In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Basic Block and CFG Classes

In [2]:
class BasicBlock:
    def __init__(self, name):
        self.name = name                  # Block label or address
        self.instructions = []           # List of instructions
        self.successors = set()          # Edges to next blocks
        self.predecessors = set()        # Incoming edges
        self.execution_count = 0          # Number of times this block was executed
        self.execution_time = 0          # Time taken to execute this block

class ControlFlowGraph:
    def __init__(self):
        self.blocks = {}                 # name -> BasicBlock

    def add_block(self, block):
        self.blocks[block.name] = block

    def add_edge(self, from_block, to_block):
        self.blocks[from_block].successors.add(to_block)
        self.blocks[to_block].predecessors.add(from_block)


## Generating CFG Data

In [3]:
# List all binaries in the profiles/embench directory
binaries_dir = "profiles/embench"
binaries = [f for f in os.listdir(binaries_dir) if os.path.isfile(os.path.join(binaries_dir, f))]

# Run the command for each binary
for binary in binaries:
    binary_path = os.path.join(binaries_dir, binary)
    command = f"./profiles/cfg_gen {binary_path}"
    os.system(command)

Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.
Benchmark completed successfully with no errors.


## Extracting CFG Data from file into CFG class 

In [28]:
def extract_cfg(benchmark):
    lines = []

    with open(f"profiles/cfgs/{benchmark}.dot", "r") as f:
        lines = f.readlines()
    
    if len(lines) == 0:
        print(f"Error: No lines found in {benchmark}.dot")
        return None

    df = pd.read_csv(f"profiles/dataset_full/{benchmark}_dataset.csv")
    if df.empty:
        print(f"Error: No data found in {benchmark}_dataset.csv")
        return None

    # Parse the .dot file to extract basic blocks and edges
    cfg = ControlFlowGraph()
    for line in lines:
        if "->" in line:
            parts = line.strip().split(" -> ")
            from_block = parts[0].strip().replace("\"","")
            # Remove the label from the block name
            to_block = parts[1].strip().split(";")[0].strip().replace("\"","").split("[")[0].strip()

            # Create blocks if they don't exist
            if from_block not in cfg.blocks:
                cfg.add_block(BasicBlock(from_block))
                
                row = df[df["basic_block_address"] == from_block]
                if not row.empty:
                    # Update the execution count and time for the block
                    block = cfg.blocks[from_block]
                    block.execution_count += row["times_executed"].values[0]
                    block.execution_time += row["time"].values[0]

            if to_block not in cfg.blocks:
                cfg.add_block(BasicBlock(to_block))
                # Add edge between blocks   
                row = df[df["basic_block_address"] == to_block]
                if not row.empty:
                    # Update the execution count and time for the block
                    block = cfg.blocks[to_block]
                    block.execution_count += row["times_executed"].values[0]
                    block.execution_time += row["time"].values[0]
            
            cfg.add_edge(from_block, to_block)
            # print row from df where "block" column is from_block
    return cfg

In [57]:
cfg = extract_cfg("aha-mont64")  # Replace with the actual benchmark name
# print blocks in CFG
with open("cfg_blocks_output.txt", "w") as file:
    for block_name, block in cfg.blocks.items():
        file.write(f"Block: {block_name}, Execution Count: {block.execution_count}, Execution Time: {block.execution_time}\n")
        file.write(f"Successors: {block.successors}\n")
        file.write(f"Predecessors: {block.predecessors}\n\n")
        print(f"Block: {block_name}, Execution Count: {block.execution_count}, Execution Time: {block.execution_time}")
        print(f"Successors: {block.successors}")
        print(f"Predecessors: {block.predecessors}")
        print()


Block: 10c38, Execution Count: 1, Execution Time: 140.0
Successors: {'15cf8'}
Predecessors: {'1392c'}

Block: 15cf8, Execution Count: 0, Execution Time: 0
Successors: {'END'}
Predecessors: {'10c38'}

Block: 13914, Execution Count: 3, Execution Time: 47.333333333333336
Successors: {'138f8', '13924'}
Predecessors: {'158b8'}

Block: 13924, Execution Count: 1, Execution Time: 140.0
Successors: {'1392c'}
Predecessors: {'13914'}

Block: 15900, Execution Count: 1, Execution Time: 139.0
Successors: {'1587c'}
Predecessors: {'14790'}

Block: 1587c, Execution Count: 3, Execution Time: 74.66666666666667
Successors: {'15898'}
Predecessors: {'15870', '15900'}

Block: 13220, Execution Count: 1, Execution Time: 130.0
Successors: {'130d8'}
Predecessors: {'131f8'}

Block: 130d8, Execution Count: 1, Execution Time: 137.0
Successors: {'14790'}
Predecessors: {'13220'}

Block: 1478c, Execution Count: 2, Execution Time: 199.0
Successors: {'13014', '13cc0'}
Predecessors: {'13cbc', '12ff8'}

Block: 13014, Exec

In [152]:
def extract_all_single_entry_exit_sequences(cfg):
    sese_regions = []

    for block_name in cfg.blocks:
        start = cfg.blocks[block_name]
        region = set()
        stack = [start.name]
        region_entry = {start.name}

        while stack:
            current = stack.pop()
            if current in region:
                continue
            region.add(current)

            block = cfg.blocks[current]

            for succ in block.successors:
                succ_block = cfg.blocks[succ]

                # Stop if successor has a predecessor outside the region
                external_preds = [
                    pred for pred in succ_block.predecessors if pred not in region
                ]
                if len(external_preds) > 0 and succ not in region:
                    continue  # not a clean single-entry region

                stack.append(succ)

        # Identify entry and exit blocks of the region
        entry_candidates = [
            b for b in region if any(p not in region for p in cfg.blocks[b].predecessors)
        ]
        exit_candidates = [
            b for b in region if any(s not in region for s in cfg.blocks[b].successors)
        ]

        if len(entry_candidates) == 1 and len(exit_candidates) == 1 and len(region) > 1:
            sese_regions.append(region)

    return sese_regions


In [153]:
sese_regions = extract_all_single_entry_exit_sequences(cfg)
# Print the single-entry single-exit regions
with open("sese_regions_output.txt", "w") as file:
    for i, region in enumerate(sese_regions):
        output = f"Region {i + 1}:\n"
        for block_name in region:
            block = cfg.blocks[block_name]
            output += f"  Block: {block_name}, Execution Count: {block.execution_count}, Execution Time: {block.execution_time}\n"
        output += "\n"
        print(output)
        file.write(output)

Region 1:
  Block: 138f8, Execution Count: 2, Execution Time: 89.5
  Block: 1392c, Execution Count: 1, Execution Time: 140.0
  Block: 15cf8, Execution Count: 0, Execution Time: 0
  Block: 13924, Execution Count: 1, Execution Time: 140.0
  Block: 13914, Execution Count: 3, Execution Time: 47.333333333333336
  Block: 10c38, Execution Count: 1, Execution Time: 140.0
  Block: END, Execution Count: 0, Execution Time: 0


Region 2:
  Block: 138f8, Execution Count: 2, Execution Time: 89.5
  Block: 158ac, Execution Count: 3, Execution Time: 59.333333333333336
  Block: 158b0, Execution Count: 3, Execution Time: 56.333333333333336
  Block: 1392c, Execution Count: 1, Execution Time: 140.0
  Block: 1587c, Execution Count: 3, Execution Time: 74.66666666666667
  Block: 13924, Execution Count: 1, Execution Time: 140.0
  Block: 15cf8, Execution Count: 0, Execution Time: 0
  Block: 158b8, Execution Count: 3, Execution Time: 47.333333333333336
  Block: 13914, Execution Count: 3, Execution Time: 47.33333

In [99]:
def compute_dominators(cfg):
    """
    Compute dominators for each block in the CFG.
    Returns a dict mapping block name to the set of block names that dominate it.
    Assumes a unique entry block (a block with no predecessors).
    """
    # Identify unique entry block (block with no predecessors)
    entry_blocks = [b for b in cfg.blocks.values() if not b.predecessors]
    if not entry_blocks:
        raise ValueError("CFG does not have an entry block (no block without predecessors)")
    entry = entry_blocks[0]

    # Initialize: every block is initially dominated by all blocks.
    dom = {b.name: set(cfg.blocks.keys()) for b in cfg.blocks.values()}
    dom[entry.name] = {entry.name}

    changed = True
    while changed:
        changed = False
        # Process every block except the entry
        for b in cfg.blocks.values():
            if b == entry:
                continue
            # Intersection of dominators of all predecessors
            pred_doms = [dom[pred] for pred in b.predecessors]
            # If a block has no predecessor (disconnected), skip update.
            if not pred_doms:
                continue
            new_dom = set.intersection(*pred_doms)
            new_dom.add(b.name)
            if new_dom != dom[b.name]:
                dom[b.name] = new_dom
                changed = True
    return dom


def compute_post_dominators(cfg):
    """
    Compute post-dominators for each block in the CFG.
    Returns a dict mapping block name to the set of block names that post-dominate it.
    Assumes a unique exit block (a block with no successors).
    """
    # Identify unique exit block (block with no successors)
    exit_blocks = [b for b in cfg.blocks.values() if not b.successors]
    if not exit_blocks:
        raise ValueError("CFG does not have an exit block (no block without successors)")
    # For simplicity we assume there is one exit block.
    exit_block = exit_blocks[0]

    # Initialize: every block is initially post-dominated by all blocks.
    pdom = {b.name: set(cfg.blocks.keys()) for b in cfg.blocks.values()}
    pdom[exit_block.name] = {exit_block.name}

    changed = True
    while changed:
        changed = False
        for b in cfg.blocks.values():
            if b == exit_block:
                continue
            # Intersection over successors’ post-dominators
            succ_pdoms = [pdom[succ] for succ in b.successors]
            if not succ_pdoms:
                continue
            new_pdom = set.intersection(*succ_pdoms)
            new_pdom.add(b.name)
            if new_pdom != pdom[b.name]:
                pdom[b.name] = new_pdom
                changed = True
    return pdom


def get_immediate_post_dominator(block, pdom):
    """
    Given a block and the post-dominator mapping, return the immediate post-dominator (ipd)
    of block. This is chosen among all post-dominators of block (except block itself)
    as the candidate that is 'closest' to block.
    """
    # Candidate post dominators (exclude self)
    candidates = pdom[block.name] - {block.name}
    if not candidates:
        return None
    # A simple way: choose the candidate for which no other candidate is strictly between it and block.
    # For each candidate, check that none of the others is a post-dominator of the candidate.
    for cand in candidates:
        if all(cand == other or cand not in pdom[other] for other in candidates):
            return cand
    return None


def extract_all_single_entry_exit_sequences(cfg):
    """
    For a given control flow graph (cfg) extract all single-entry single-exit (SESE) regions.
    
    Returns a list of tuples (entry_name, exit_name, region_set) where:
      - entry_name: name of the entry block of the region,
      - exit_name: name of the exit block,
      - region_set: a set of block names constituting the region.
    
    A region is identified by choosing a block's immediate post-dominator as the exit.
    Then the region is defined as the set of blocks n for which:
       - entry_name is in dom[n]
       - exit_name is in pdom[n]
    Finally, we validate that the region has exactly one entry and one exit.
    """
    # Compute dominators and post-dominators.
    dom = compute_dominators(cfg)
    pdom = compute_post_dominators(cfg)
    
    sese_regions = []
    
    # For every block, try to get a region with it as entry.
    for b in cfg.blocks.values():
        ipd = get_immediate_post_dominator(b, pdom)
        if ipd is None:
            continue  # cannot form a region without an exit candidate
        
        entry_name = b.name
        exit_name = ipd  # immediate post-dominator candidate
        
        # Define the candidate region as all blocks for which:
        # - entry_name dominates that block, and
        # - exit_name post-dominates that block.
        region = {name for name in cfg.blocks.keys() if entry_name in dom[name] and exit_name in pdom[name]}
        
        # Validation:
        #   * The region should have a single entry: only one block inside the region should have a predecessor not in region.
        #   * And similarly, only one block (the exit) should have a successor outside the region.
        entry_candidates = [
            name for name in region
            if any(pred not in region for pred in cfg.blocks[name].predecessors)
        ]
        exit_candidates = [
            name for name in region
            if any(succ not in region for succ in cfg.blocks[name].successors)
        ]
        
        if len(entry_candidates) == 1 and len(exit_candidates) == 1:
            sese_regions.append((entry_candidates[0], exit_candidates[0], region))
    
    return sese_regions


In [101]:
# # Example usage (if you want to try it out):

# if __name__ == "__main__":
#     # Create a simple CFG.
#     cfg = ControlFlowGraph()
    
#     # Construct basic blocks.
#     A = BasicBlock("A")
#     B = BasicBlock("B")
#     C = BasicBlock("C")
#     D = BasicBlock("D")
#     E = BasicBlock("E")
    
#     # Add blocks to the CFG.
#     for block in [A, B, C, D, E]:
#         cfg.add_block(block)
    
#     # Set up edges. For example, a diamond-shaped structure:
#     #   A -> B, A -> C, B -> D, C -> D, D -> E
#     cfg.add_edge("A", "B")
#     cfg.add_edge("A", "C")
#     cfg.add_edge("B", "D")
#     cfg.add_edge("C", "D")
#     cfg.add_edge("D", "E")
    
# Extract SESE regions.
regions = extract_all_single_entry_exit_sequences(cfg)
for entry, exit, region in regions:
    print(f"SESE Region: Entry = {entry}, Exit = {exit}, Blocks = {sorted(region)}")

SESE Region: Entry = 10c38, Exit = 15cf8, Blocks = ['10c38', '15cf8']
SESE Region: Entry = 13924, Exit = 1392c, Blocks = ['13924', '1392c']
SESE Region: Entry = 15900, Exit = 15900, Blocks = ['15900']
SESE Region: Entry = 13220, Exit = 130d8, Blocks = ['130d8', '13220']
SESE Region: Entry = 130d8, Exit = 130d8, Blocks = ['130d8']
SESE Region: Entry = 13014, Exit = 131f8, Blocks = ['13014', '131f8']
SESE Region: Entry = 12ff8, Exit = 12ff8, Blocks = ['12ff8']
SESE Region: Entry = 127b8, Exit = 127b8, Blocks = ['127b8']
SESE Region: Entry = 138f8, Exit = 138f8, Blocks = ['138f8']
SESE Region: Entry = 157c0, Exit = 1586c, Blocks = ['157c0', '1586c']
SESE Region: Entry = 1586c, Exit = 15870, Blocks = ['1586c', '15870']
SESE Region: Entry = 15ca0, Exit = 15cd0, Blocks = ['15ca0', '15cd0']
SESE Region: Entry = 15cd0, Exit = 157b8, Blocks = ['157b8', '15cd0']
SESE Region: Entry = 1526c, Exit = 15794, Blocks = ['1526c', '15794']
SESE Region: Entry = 15794, Exit = 15ca0, Blocks = ['15794', '15c

In [45]:
def extract_all_single_entry_exit_sequences(cfg):
    sequences = []

    for start in cfg.blocks.values():
        visited = set()
        stack = [start]
        region = set()
        entry_block = start
        exit_blocks = set()

        while stack:
            current = stack.pop()
            if current.name in visited:
                continue
            visited.add(current.name)
            region.add(current.name)

            for succ_name in current.successors:
                succ = cfg.blocks[succ_name]
                # If successor has a predecessor outside the region, it’s a potential exit
                if any(pred not in region for pred in succ.predecessors):
                    exit_blocks.add(succ_name)
                else:
                    stack.append(succ)

        # Filter to ensure only one entry and one exit
        external_preds = set()
        external_succs = set()
        for name in region:
            block = cfg.blocks[name]
            for pred in block.predecessors:
                if pred not in region:
                    external_preds.add(pred)
            for succ in block.successors:
                if succ not in region:
                    external_succs.add(succ)

        if len(external_preds) == 1 and len(external_succs) == 1:
            sequences.append(region)

    # Deduplicate overlapping regions (optional)
    unique_sequences = []
    seen = set()
    for seq in sequences:
        frozen = frozenset(seq)
        if frozen not in seen:
            unique_sequences.append(seq)
            seen.add(frozen)

    return unique_sequences


In [100]:
regions = extract_all_single_entry_exit_sequences(cfg)

with open("sese_regions_output.txt", "w") as file:
    for i, region in enumerate(regions):
        output = f"Region {i + 1}:\n"
        for block_name in region:
            block = cfg.blocks[block_name]
            output += f"  Block: {block_name}, Execution Count: {block.execution_count}, Execution Time: {block.execution_time}\n"
        output += "\n"
        print(output)
        file.write(output)

TypeError: unhashable type: 'set'

In [ ]:
def extract_all_single_entry_exit_sequences(cfg):
    sese_regions = []

    for block_name in cfg.blocks:
        start = cfg.blocks[block_name]
        region = set()
        stack = [start.name]
        region_entry = {start.name}

        while stack:
            current = stack.pop()
            if current in region:
                continue
            region.add(current)

            block = cfg.blocks[current]
            legal = True

            for succ in block.successors:
                succ_block = cfg.blocks[succ]

                # Stop if successor has a predecessor outside the region
                external_preds = [
                    pred for pred in succ_block.predecessors if pred not in region 
                    and pred not in succ_block.successors
                ]
                if len(external_preds) > 0 and succ not in region:
                    legal = False
            if legal:
                sese_regions.append(region)
                for succ in block.successors:
                    stack.append(succ)

        # Identify entry and exit blocks of the region
        entry_candidates = [
            b for b in region if any(p not in region for p in cfg.blocks[b].predecessors)
        ]
        exit_candidates = [
            b for b in region if any(s not in region for s in cfg.blocks[b].successors)
        ]

        if len(entry_candidates) == 1 and len(exit_candidates) == 1 and len(region) > 1:
        # if len(region) > 1:
            sese_regions.append(region)

    return sese_regions


In [143]:
regions = extract_all_single_entry_exit_sequences(cfg)

with open("sese_regions_output.txt", "w") as file:
    for i, region in enumerate(regions):
        output = f"Region {i + 1}:\n"
        for block_name in region:
            block = cfg.blocks[block_name]
            output += f"  Block: {block_name}, Execution Count: {block.execution_count}, Execution Time: {block.execution_time}\n"
        output += "\n"
        print(output)
        file.write(output)

Region 1:
  Block: 10c38, Execution Count: 1, Execution Time: 140.0
  Block: END, Execution Count: 0, Execution Time: 0
  Block: 15cf8, Execution Count: 0, Execution Time: 0


Region 2:
  Block: 138f8, Execution Count: 2, Execution Time: 89.5
  Block: 1392c, Execution Count: 1, Execution Time: 140.0
  Block: 15cf8, Execution Count: 0, Execution Time: 0
  Block: 13924, Execution Count: 1, Execution Time: 140.0
  Block: 13914, Execution Count: 3, Execution Time: 47.333333333333336
  Block: 10c38, Execution Count: 1, Execution Time: 140.0
  Block: END, Execution Count: 0, Execution Time: 0


Region 3:
  Block: 138f8, Execution Count: 2, Execution Time: 89.5
  Block: 1392c, Execution Count: 1, Execution Time: 140.0
  Block: 15cf8, Execution Count: 0, Execution Time: 0
  Block: 13924, Execution Count: 1, Execution Time: 140.0
  Block: 13914, Execution Count: 3, Execution Time: 47.333333333333336
  Block: 10c38, Execution Count: 1, Execution Time: 140.0
  Block: END, Execution Count: 0, Exec